In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('font', family='NanumGothic')
matplotlib.rcParams['axes.unicode_minus'] = False   # 마이너스 부호 깨짐

In [23]:
train_ln = pd.read_csv('./data/train_ln.csv',parse_dates=['ymd'])
train_dn = pd.read_csv('./data/train_dn.csv',parse_dates=['ymd'])
test_ln = pd.read_csv('./data/test_ln.csv',parse_dates=['ymd'])
test_dn = pd.read_csv('./data/test_dn.csv',parse_dates=['ymd'])

In [24]:
sub = pd.read_csv('./data/original/sample_submission.csv')

In [25]:
train_ln = train_ln.drop(['menu2','side2','side3','kimchi','etc'],axis=1)
train_dn = train_dn.drop(['menu3','side2','side3','kimchi'],axis=1)
test_ln = test_ln.drop(['menu2','side2','side3','kimchi','etc'],axis=1)
test_dn = test_dn.drop(['menu3','side2','side3','kimchi'],axis=1)

In [47]:
train_ln.columns

Index(['ymd', 'dow', 'employees', 'dayoff', 'bustrip', 'overtime', 'remote',
       'ln', 'rice', 'soup', 'side1', 'atoffice', 'ln_prop', 'dow_encoded',
       'y', 'm', 'd', 'y_encoded', 'm_encoded'],
      dtype='object')

In [63]:
train_ln['y'] = train_ln.ymd.dt.strftime('%Y')
train_ln['m'] = train_ln.ymd.dt.strftime('%m')

train_ln['ln_prop'] = train_ln.ln / train_ln.atoffice

y_encode = train_ln.groupby('y').ln_prop.mean()
m_encode = train_ln.groupby('m').ln_prop.mean()
dow_encode = train_ln.groupby('dow').ln_prop.mean()

train_ln['y_encoded'] = train_ln.y.replace(y_encode)
train_ln['m_encoded'] = train_ln.m.replace(m_encode)
train_ln['dow_encoded'] = train_ln.dow.replace(dow_encode)

In [62]:
train_ln = df_ch(train_ln,'ln')
train_dn = df_ch(train_dn,'dn')

In [64]:
test_ln['y'] = test_ln.ymd.dt.strftime('%Y')
test_ln['m'] = test_ln.ymd.dt.strftime('%m')
test_ln['y_encoded'] = test_ln.y.replace(y_encode)
test_ln['m_encoded'] = test_ln.m.replace(m_encode)
test_ln['dow_encoded'] = test_ln.dow.replace(dow_encode)

In [51]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [69]:
tmp_ln = train_ln[['overtime','atoffice',
                 'y_encoded', 'm_encoded','dow_encoded','ln']]

X_train, X_test, y_train, y_test = train_test_split(tmp_ln.iloc[:,:-1],
                                                    tmp_ln.iloc[:,-1],
                                                    test_size=0.3,
                                                    random_state=42)
rtree = RandomForestRegressor(criterion='mae', random_state=42)
rtree.fit(X_train, y_train)
y_pred = rtree.predict(X_test)

print("훈련 세트 정확도: {:.3f}".format(rtree.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(rtree.score(X_test, y_test)))
print("MAE: {:.3f}".format(mean_absolute_error(y_pred,y_test)))

훈련 세트 정확도: 0.958
테스트 세트 정확도: 0.754
MAE: 75.893


In [73]:
tmp_test_ln = test_ln[['overtime','atoffice',
                        'y_encoded', 'm_encoded','dow_encoded']]
ln_pred = rtree.predict(tmp_test_ln)
ln_pred = ln_pred.round(0).astype('int')
sub['중식계'] = ln_pred

In [65]:
train_dn['y'] = train_dn.ymd.dt.strftime('%Y')
train_dn['m'] = train_dn.ymd.dt.strftime('%m')

train_dn['dn_prop'] = train_dn.dn / train_dn.atoffice

y_encode = train_dn.groupby('y').dn_prop.mean()
m_encode = train_dn.groupby('m').dn_prop.mean()
dow_encode = train_dn.groupby('dow').dn_prop.mean()

train_dn['y_encoded'] = train_dn.y.replace(y_encode)
train_dn['m_encoded'] = train_dn.m.replace(m_encode)
train_dn['dow_encoded'] = train_dn.dow.replace(dow_encode)

In [66]:
test_dn['y'] = test_dn.ymd.dt.strftime('%Y')
test_dn['m'] = test_dn.ymd.dt.strftime('%m')
test_dn['y_encoded'] = test_dn.y.replace(y_encode)
test_dn['m_encoded'] = test_dn.m.replace(m_encode)
test_dn['dow_encoded'] = test_dn.dow.replace(dow_encode)

In [75]:
tmp_dn = train_dn[['overtime','atoffice',
                 'y_encoded', 'm_encoded','dow_encoded','dn']]

X_train, X_test, y_train, y_test = train_test_split(tmp_dn.iloc[:,:-1],
                                                    tmp_dn.iloc[:,-1],
                                                    test_size=0.3,
                                                    random_state=42)
rtree = RandomForestRegressor(criterion='mae', random_state=42)
rtree.fit(X_train, y_train)
y_pred = rtree.predict(X_test)

print("훈련 세트 정확도: {:.3f}".format(rtree.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(rtree.score(X_test, y_test)))
print("MAE: {:.3f}".format(mean_absolute_error(y_pred,y_test)))

훈련 세트 정확도: 0.946
테스트 세트 정확도: 0.647
MAE: 45.791


In [76]:
tmp_test_dn = test_dn[['overtime','atoffice',
                        'y_encoded', 'm_encoded','dow_encoded']]
dn_pred = rtree.predict(tmp_test_dn)
dn_pred = dn_pred.round(0).astype('int')
sub['석식계'] = dn_pred

In [78]:
sub.to_csv('./data/submission2.csv',index=False)